# Start the system

In [1]:
#1. Libraries
import psycopg2 as pg
import pandas as pd
import os
import re
import pycountry_convert as pc
import matplotlib.pyplot as plt
import datetime
import math
import numpy as np
import psycopg2.extras

In [2]:
#2. set environment variable for psycog2 (for some systems)
os.environ["PGGSSENCMODE"] = "disable"

#3. get the stored Passwords
f=open("credentials.txt", "rt")
pwd=f.readline().strip()  #di   pass 
f.close()

#4. test connections using the password
# FCUL server
#host_di = "appserver-01.alunos.di.fc.ul.pt"
#db_di = "tpd012"
#user_di = "tpd012"

#conn = pg.connect(host = host_di, database = db_di, user = user_di, password = pwd)
#conn.close()

# Local server
host = "localhost"
database="teste"
user="postgres"
password="0000"

conn = pg.connect(host=host,database=database, user=user, password=password)
conn.close()

### Read csv files

In [3]:
df = pd.read_csv("../data/listings_al.csv")
df_gdp = pd.read_csv('../data/GDP per capita (worldbank).csv')

/Users/air/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Transformation methods

In [4]:
# Check for missing values:
def is_nan(x):
    return (x != x)

def get_host_name(name):
    if is_nan(name): return "Unknown"
    return name

In [5]:
def get_host_membership_duration(date):
    if is_nan(date): return "Unknown"
    
    now = datetime.datetime.now()
    #date is a string with the date as YYYY-MM-DD
    membership_time = now.year - int(date[0:4])
    if membership_time > 10: return "Member for more than 10 years"
    if membership_time > 5: return "Member for more than 5 years"
    if membership_time > 2: return "Member for more than 2 years"
    if membership_time >= 1: return "Member for more than 1 year"
    if membership_time < 1: return "Member for less than 1 year"

Since the main focus of this analysis is based on the host location, we are not interested in keeping records lacking this information. In that sense, all host records who don't have a known location will be ignored - None lines will be later removed in preprocessing.

In [6]:
def get_host_country(location):
    #location is a string with the host location -- may not have country
    location = str(location)
    if re.search('po.*tugal', location, re.IGNORECASE): return "Portugal"
    if re.search('lisbo', location, re.IGNORECASE): return "Portugal"
    if re.search('PT', location): return "Portugal"
    if re.search('spain', location, re.IGNORECASE): return "Spain"
    if re.search('ES', location): return "Spain"
    if re.search('madrid', location, re.IGNORECASE): return "Spain"
    if re.search('united kingdom', location, re.IGNORECASE): return "United Kingdom"
    if re.search('UK', location): return "United Kingdom"
    if re.search('GB', location): return "United Kingdom"
    if re.search('ireland', location, re.IGNORECASE): return "United Kingdom"
    if re.search('denmark', location, re.IGNORECASE): return "Denmark"
    if re.search('netherlands', location, re.IGNORECASE): return "Netherlands"
    if re.search('NL', location): return "Netherlands"
    if re.search('germany', location, re.IGNORECASE): return "Germany"
    if re.search('DE', location): return "Germany"
    if re.search('belgium', location, re.IGNORECASE): return "Belgium"
    if re.search('united states', location, re.IGNORECASE): return "United States"
    if re.search('US', location): return "United States"
    if re.search('canada', location, re.IGNORECASE): return "Canada"
    if re.search('france', location, re.IGNORECASE): return "France"
    if re.search('FR', location): return "France"
    if re.search('italy', location, re.IGNORECASE): return "Italy"
    if re.search('IT', location): return "Italy"
    if re.search('switzerland', location, re.IGNORECASE): return "Switzerland"
    if re.search('sweden', location, re.IGNORECASE): return "Sweden"
    if re.search('poland', location, re.IGNORECASE): return "Poland"
    if re.search('finland', location, re.IGNORECASE): return "Finland"
    if re.search('serbia', location, re.IGNORECASE): return "Serbia"
    if re.search('austria', location, re.IGNORECASE): return "Austria"
    if re.search('iceland', location, re.IGNORECASE): return "Iceland"
    if re.search('norway', location, re.IGNORECASE): return "Norway"
    if re.search('china', location, re.IGNORECASE): return "China"
    if re.search('angola', location, re.IGNORECASE): return "Angola"
    if re.search('australia', location, re.IGNORECASE): return "Australia"
    if re.search('brazil', location, re.IGNORECASE): return "Brazil"
    if re.search('BR', location): return "Brazil"
    if re.search('peru', location, re.IGNORECASE): return "Peru"
    if re.search('sri lanka', location, re.IGNORECASE): return "Sri Lanka"
    if re.search('vietnam', location, re.IGNORECASE): return "Vietnam"
    if re.search('united arab emirates', location, re.IGNORECASE): return "United Arab Emirates"
    if re.search('south africa', location, re.IGNORECASE): return "South Africa"
    if re.search('qatar', location, re.IGNORECASE): return "Qatar"
    if re.search('japan', location, re.IGNORECASE): return "Japan"
    if re.search('turkey', location, re.IGNORECASE): return "Turkey"
    else: return None

One thing to keep in mind is that this method of identificating the host country is not the best, since it is not very generalizable to new data. In order for this method to work safely for all new data, an extended 'database' of countries would be needed. Here we made an effort to cover the most common countries in the dataset, but did not extend it much in order to preserve performance.

One alternative that we pursued was using the `pycountry` library, but since it ws very inefficient (for each record it would go through all countries in the library) we decided that for this purpose the purpose of this work the method above could be satisfatory.

In [7]:
def get_host_continent(country):
    if country == None: return None
    
    continents_dic = {
    'EU': 'Europe',
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    }
    
    # Acquire the country code
    country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
    # Convert country code to continent code; use dictionary to return continent name
    continent_name = pc.country_alpha2_to_continent_code(country_code)
    
    return continents_dic[continent_name]

In [8]:
def get_host_country_gdp(country):
    if country == None: return None
    
    # Always get latest (most updated) gdp available in the gdp dataset - last column
    # The level of detail required here is not very high, since we will use bins
    # to characterize the countries' wealth
    dict_gdp = pd.Series(df_gdp.iloc[:,-1].values, index=df_gdp['Country Name']).to_dict()
    
    if country not in dict_gdp.keys(): return "Unknown"
    
    gdp = dict_gdp[country]
    if gdp <= 20000  : return "Below 20k"
    if gdp <= 35000  : return "20k - 35k"
    if gdp <= 50000  : return "35k - 50k"
    if gdp > 50000 : return "Above 50k"
    else: return None

In [9]:
def get_host_response_time(response_time):
    if is_nan(response_time): return "Unknown"
    return response_time

In [10]:
def get_host_is_superhost(is_superhost):
    if is_superhost == 't': return "Superhost"
    if is_superhost == 'f': return "Not Superhost"
    else: return None

In [11]:
def get_host_identity_verified(identity):
    if identity == 't': return "Verified"
    if identity == 'f': return "Unverified"
    else: return None

### Create clean dataframe

In [61]:
listing_id = [i for i in df['id']]
names = [n for n in df['host_name']]
membership_duration = [get_host_membership_duration(d) for d in df['host_since']]
country = [get_host_country(l) for l in df['host_location']]
continent = [get_host_continent(c) for c in country]
gdp = [get_host_country_gdp(c) for c in country]
response_time = [get_host_response_time(t) for t in df['host_response_time']]
superhost = [get_host_is_superhost(b) for b in df['host_is_superhost']]
identity_verified = [get_host_identity_verified(i) for i in df['host_identity_verified']]

In [62]:
columns = ['listing_id','host_name', 'membership_duration','host_country', 'host_continent', 
           'host_country_gdp', 'host_response_time', 'host_is_superhost', 'host_is_verified']
df_host_dimension = pd.DataFrame(np.stack((listing_id, names, membership_duration, country, 
                                           continent, gdp, response_time, superhost, 
                                           identity_verified),axis=-1), columns = columns)
df_host_dimension.shape

(17168, 9)

In [63]:
def delete_null_rows(df,columns):
    """Removes all records with any empty cells from input DataFrame"""
    processed_df = df[columns].copy()
    total_rows = processed_df.shape[0]
    delete_rows = []
    
    if processed_df.isnull().values.any(): # if there are any null values in DataFrame, process DataFrame
        for index, row in enumerate(processed_df.itertuples(), start = 0):
            if (pd.Series(row).isnull().values.any()): # if row has any null value
                delete_rows.append(index) # add row index to delete list
    
    processed_df.drop(df.index[delete_rows], inplace = True) # delete rows fr
    processed_df = processed_df.reset_index().drop('index', axis = 1)
    print('DataFrame contains {} rows. Deleted {} rows ({}% of total rows)'.format(processed_df.shape[0], len(delete_rows), round(len(delete_rows)*100/total_rows, 2)))
    return processed_df

In [64]:
df_host_dimension = delete_null_rows(df_host_dimension, columns)

DataFrame contains 17005 rows. Deleted 163 rows (0.95% of total rows)


Add primary key:

In [65]:
pks = [i for i in range(1,df_host_dimension.shape[0]+1)]
df_host_dimension.insert(0, "host_id", pks, True) 

Remove duplicates, mapping the each listing to these new FK.

In [ ]:
columns = ['host_id','host_name', 'membership_duration','host_country', 'host_continent', 
           'host_country_gdp', 'host_response_time', 'host_is_superhost', 'host_is_verified']

host_dimension_processed = df_host_dimension[columns].drop_duplicates().copy()
host_dimension_processed = host_dimension_processed.reset_index().drop('index', axis = 1)
host_dimension_processed.index += 1

host_dimension_processed

## Attributing dimension keys to facts

In [ ]:
# match fact IDs with FKs in dimension
FK = []
for i, row in enumerate(df_host_dimension.itertuples(), start = 1):
    row = list(row)
    for j, dim_row in enumerate(host_dimension_processed.itertuples(), start = 1):
        dim_row = list(dim_row)
        if (row[2:] == dim_row[1:]): # if fact attributes are equal to dimension record attributes
            FK.append(j)
            break # leaves nested loop if match is found  
len(FK)

In [ ]:
df_mapping = pd.DataFrame({})
df_mapping['ID'] = list(df_host_dimension['listing_id'].copy())
df_mapping['host_id'] = FK
df_mapping.index += 1

In [ ]:
df_mapping

### Load Host dimension

In [19]:
delete_table = """
DROP TABLE IF EXISTS Host;
"""

create_table = """
CREATE TABLE Host (
    HOST_ID SERIAL PRIMARY KEY NOT NULL,
    HOST_NAME VARCHAR(40) NOT NULL,
    MEMBERSHIP_DURATION VARCHAR(40) CHECK (MEMBERSHIP_DURATION in ('Member for more than 10 years','Member for more than 5 years','Member for more than 2 years','Member for more than 1 year','Member for less than 1 year', 'Unknown')) NOT NULL,
    HOST_COUNTRY VARCHAR(20) NOT NULL,
    HOST_CONTINENT VARCHAR(20) CHECK (HOST_CONTINENT in ('Europe','North America','South America','Asia','Australia','Africa')) NOT NULL,
    HOST_COUNTRY_GDP VARCHAR(20) CHECK (HOST_COUNTRY_GDP in ('Below 20k','20k - 35k','35k - 50k','Above 50k')) NOT NULL,
    HOST_RESPONSE_TIME VARCHAR(20) CHECK (HOST_RESPONSE_TIME in ('within an hour','within a few hours','within a day','a few days or more','Unknown')) NOT NULL,
    HOST_IS_SUPERHOST VARCHAR(20) CHECK (HOST_IS_SUPERHOST in ('Superhost','Not Superhost')) NOT NULL,
    HOST_IS_VERIFIED VARCHAR(20) CHECK (HOST_IS_VERIFIED in ('Verified','Unverified')) NOT NULL
    )
"""

In [20]:
def excuteSingleSQLstatement(sql, host, database, user, password):
    conn = pg.connect(host=host,database=database, user=user, password=password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

In [21]:
excuteSingleSQLstatement(delete_table, host, database, user, password)
excuteSingleSQLstatement(create_table, host, database, user, password)

In [22]:
#method for bulk insert
def insert_data(df, table_name, conn):
    df_columns = list(df)
    columns = ",".join(df_columns)
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
    insert_stmt = "INSERT INTO {} ({}) {}".format(table_name,columns,values)
    success = True
    try:
        cursor = conn.cursor()
        pg.extras.execute_batch(cursor, insert_stmt, df.values)
        conn.commit()
        success = True
    except pg.DatabaseError as error:
        success = False
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return success

In [23]:
if len(df_host_dimension) > 0:
    table_name = 'Host'
    conn = pg.connect(host=host, database=database, user=user, password=password)
    success = insert_data(df_host_dimension,table_name, conn)
    conn.close()
    if success == True: print('Data inserted succefully')
else: print('No data to insert')

Data inserted succefully
